In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
import re
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import *
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = os.getcwd()
path

'/Users/adamroth/Documents/flat_fe_work/mod4/week13/NLP---Twitter-Sentiment/notebooks/AR_notebooks'

### basic info

In [3]:
raw_data = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')

In [4]:
raw_data.sample(15)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
4323,Srsly? ÛÏ@mention It's not a rumor: Apple is ...,iPad,Positive emotion
2928,Next presentation - Google's Marissa Mayer. Sh...,Google,Positive emotion
7177,It just got real. Apple Store #SXSW {link},NaN,No emotion toward brand or product
4308,If only @mention brought up the Google Whiteli...,NaN,No emotion toward brand or product
7555,&quot;O frabjous day ! Callooh ! Callay !Û_[B...,iPad,Positive emotion
6779,RT @mention Theres a class war forming here at...,NaN,No emotion toward brand or product
5890,RT @mention Google to Launch Major New Social ...,NaN,No emotion toward brand or product
6131,RT @mention In my next life I'm coming back as...,iPad,Positive emotion
1441,Confirmed! Apple store 2 week popup in Austin ...,NaN,No emotion toward brand or product
2622,Short line at the Apple pop up store in Austin...,Apple,Positive emotion


In [5]:
raw_data['emotion_in_tweet_is_directed_at'].sample(10)

8048       NaN
4111     Apple
7159    Google
3378       NaN
2157     Apple
3385     Apple
7975       NaN
1411       NaN
8481       NaN
7396       NaN
Name: emotion_in_tweet_is_directed_at, dtype: object

In [6]:
raw_data['emotion_in_tweet_is_directed_at'].unique()

array(['iPhone', 'iPad or iPhone App', 'iPad', 'Google', nan, 'Android',
       'Apple', 'Android App', 'Other Google product or service',
       'Other Apple product or service'], dtype=object)

In [7]:
raw_data.isna()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
9088,False,False,False
9089,False,True,False
9090,False,True,False
9091,False,True,False


In [8]:
raw_data.dtypes

tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object

In [9]:
raw_data['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [10]:
#lots of nan's in this column, shouldn't matter while training model though

In [11]:
raw_data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [12]:
#not many negatives, might need to mess with that, could use something like smote, might remove all from 'I cant tell'

### Tokenization

In [13]:
raw_data.iloc[i].tweet_text

NameError: name 'i' is not defined

In [14]:
raw_data.iloc[0].tweet_text.split()

['.@wesley83',
 'I',
 'have',
 'a',
 '3G',
 'iPhone.',
 'After',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#RISE_Austin,',
 'it',
 'was',
 'dead!',
 'I',
 'need',
 'to',
 'upgrade.',
 'Plugin',
 'stations',
 'at',
 '#SXSW.']

In [15]:
test_list = raw_data.iloc[0].tweet_text.split()
type(test_list)

list

In [16]:
df = raw_data
df['split_text'] = 0
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product', 'split_text'],
      dtype='object')

In [17]:
raw_data['tweet_text'][5]

'@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd'

In [18]:
for i in range(0,len(raw_data)):
    text = raw_data['tweet_text'][i]
    split_text = str(text).split()
    df['split_text'][i] = split_text
    
    



/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
df['split_text'][4]

['@sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 '#SXSW:',
 'Marissa',
 'Mayer',
 '(Google),',
 'Tim',
 "O'Reilly",
 '(tech',
 'books/conferences)',
 '&amp;',
 'Matt',
 'Mullenweg',
 '(Wordpress)']

In [20]:
import string
punctuation = string.punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
def clean_punct(string):

    clean_text_list = [''.join(ch for ch in s if ch not in punctuation) for s in string]
    
    #clean_text = ""
    
    #return (clean_text.join(clean_text_list))
    return clean_text_list
#cleans up punctuation

In [22]:
string = "@sxtxstate"
clean_punct(string)
print(clean_punct(string))

['', 's', 'x', 't', 'x', 's', 't', 'a', 't', 'e']


In [23]:
clean_punct(df['split_text'][4])

['sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 'SXSW',
 'Marissa',
 'Mayer',
 'Google',
 'Tim',
 'OReilly',
 'tech',
 'booksconferences',
 'amp',
 'Matt',
 'Mullenweg',
 'Wordpress']

In [24]:
# def clean_column(df['column']):
#     for i in range(0, len(df)):
#         thing = df['column'][i]
#         clean_string = str(thing)
#         return 

In [25]:
df['no_punct'] = 0

In [26]:
for i in range(0, len(df)):
    has_punct = df['split_text'][i]
    no_punct = clean_punct(has_punct)
    df['no_punct'][i] = no_punct

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [27]:
df['no_punct'][1]

['jessedee',
 'Know',
 'about',
 'fludapp',
 '',
 'Awesome',
 'iPadiPhone',
 'app',
 'that',
 'youll',
 'likely',
 'appreciate',
 'for',
 'its',
 'design',
 'Also',
 'theyre',
 'giving',
 'free',
 'Ts',
 'at',
 'SXSW']

In [28]:
df['clean'] = 0
df['clean'].head()

0    0
1    0
2    0
3    0
4    0
Name: clean, dtype: int64

In [29]:
for i in range(0, len(df)):
    a = df['no_punct'][i]
    now_lower = str(a).lower()
    df['clean'][i] = now_lower
#I'm a dumb dumb who forgot that df.apply is a thing

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
df['clean'].head()

0    ['wesley83', 'i', 'have', 'a', '3g', 'iphone',...
1    ['jessedee', 'know', 'about', 'fludapp', '', '...
2    ['swonderlin', 'can', 'not', 'wait', 'for', 'i...
3    ['sxsw', 'i', 'hope', 'this', 'years', 'festiv...
4    ['sxtxstate', 'great', 'stuff', 'on', 'fri', '...
Name: clean, dtype: object

In [31]:
print(len(df))
df.sample(10)

9093


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,split_text,no_punct,clean
2255,Going to be awesome! RT @mention Gallery of or...,NaN,No emotion toward brand or product,"[Going, to, be, awesome!, RT, @mention, Galler...","[Going, to, be, awesome, RT, mention, Gallery,...","['going', 'to', 'be', 'awesome', 'rt', 'mentio..."
5511,RT @mention At #sxsw? @mention / @mention wann...,NaN,No emotion toward brand or product,"[RT, @mention, At, #sxsw?, @mention, /, @menti...","[RT, mention, At, sxsw, mention, , mention, wa...","['rt', 'mention', 'at', 'sxsw', 'mention', '',..."
1045,ÛÏ@mention RT @mention Google to Launch New S...,NaN,No emotion toward brand or product,"[ÛÏ@mention, RT, @mention, Google, to, Launch...","[ÛÏmention, RT, mention, Google, to, Launch, ...","['\x89ûïmention', 'rt', 'mention', 'google', '..."
4425,My post on Google/Bing Q/A with @mention @ment...,NaN,No emotion toward brand or product,"[My, post, on, Google/Bing, Q/A, with, @mentio...","[My, post, on, GoogleBing, QA, with, mention, ...","['my', 'post', 'on', 'googlebing', 'qa', 'with..."
3121,#sxsw is exposing my iphone's horrendous batte...,iPhone,Negative emotion,"[#sxsw, is, exposing, my, iphone's, horrendous...","[sxsw, is, exposing, my, iphones, horrendous, ...","['sxsw', 'is', 'exposing', 'my', 'iphones', 'h..."
4298,yup. RT @mention @mention at #sxsw: &quot;appl...,Apple,Positive emotion,"[yup., RT, @mention, @mention, at, #sxsw:, &qu...","[yup, RT, mention, mention, at, sxsw, quotappl...","['yup', 'rt', 'mention', 'mention', 'at', 'sxs..."
7301,just a tip for @mention iphone users. turn dow...,NaN,No emotion toward brand or product,"[just, a, tip, for, @mention, iphone, users., ...","[just, a, tip, for, mention, iphone, users, tu...","['just', 'a', 'tip', 'for', 'mention', 'iphone..."
8168,AGREED! RT @mention If u could afford to atten...,NaN,No emotion toward brand or product,"[AGREED!, RT, @mention, If, u, could, afford, ...","[AGREED, RT, mention, If, u, could, afford, to...","['agreed', 'rt', 'mention', 'if', 'u', 'could'..."
3179,No ipad2 for me. Now I have 3 weeks w no iPad ...,NaN,Positive emotion,"[No, ipad2, for, me., Now, I, have, 3, weeks, ...","[No, ipad2, for, me, Now, I, have, 3, weeks, w...","['no', 'ipad2', 'for', 'me', 'now', 'i', 'have..."
5269,RT @mention #sxsw #csr reporting: Janecek: Mic...,Apple,Negative emotion,"[RT, @mention, #sxsw, #csr, reporting:, Janece...","[RT, mention, sxsw, csr, reporting, Janecek, M...","['rt', 'mention', 'sxsw', 'csr', 'reporting', ..."


In [42]:
sw = stopwords

In [43]:
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)


In [32]:
df_no_emo = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'No emotion toward brand or product']
print(len(df_no_emo))


5389


In [33]:
df_neg = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion']
len(df_neg)

570

In [34]:
df_pos = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion']
len(df_pos)

2978

In [35]:
df_unclear = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'I can\'t tell']
print(len(df_unclear))
df_unclear['clean'][:5]

156


90     ['thanks', 'to', 'mention', 'for', 'publishing...
102    ['\x89ûïmention', 'quotapple', 'has', 'opened'...
237    ['just', 'what', 'america', 'needs', 'rt', 'me...
341    ['the', 'queue', 'at', 'the', 'apple', 'store'...
368    ['hope', 'its', 'better', 'than', 'wave', 'rt'...
Name: clean, dtype: object

In [36]:
X = df_pos.clean.astype('str')
y = df.is_there_an_emotion_directed_at_a_brand_or_product

<WordListCorpusReader in '/Users/adamroth/nltk_data/corpora/stopwords'>